<a id="top"></a>

# Characterizing M dwarf flares with MAST: Downloading data

***

# Learning goals
By the end of this tutorial, you will:
- Understand how to find high proper motion targets with HST.
- Be able to extract target information from downloaded FITS headers.
- Understand the difference between instruments' FITS header structures.

# Introduction
The flare rate of M dwarfs — the lowest-mass stars that undergo nuclear fusion — has a broad class of science cases. Flare rates have ramifications for stellar physics (e.g., connections to stellar mass and rotation period) and exoplanet science (e.g., photochemistry and habitability). Time-series MAST data can be used to capture and constrain flares. This tutorial begins a series exploring how to use MAST data for such purposes, starting with querying and downloading Hubble Space Telescope ([HST](https://archive.stsci.edu/missions-and-data/hst)) data for M dwarfs. The series will continue on to demonstrate how to connect these HST data to shorter-cadence data from, e.g., the Transiting Exoplanet Survey Satellite ([TESS](https://archive.stsci.edu/missions-and-data/tess)).

# Imports
- numpy to help aggregate our metadata.
- pandas to sort through our list of M dwarfs.
- sys (Python builtin) to read our machine's Python version.
- astroquery.mast to interact with MAST data products.
- astropy to provide data structures that organize our metadata, units functionality, and coordinates functionality.

In [3]:
import pandas as pd

from astropy.coordinates import position_angle

import matplotlib.pyplot as plt

from urllib.error import HTTPError

import astropy.units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord


from astroquery.mast import Observations
import numpy as np

import astropy.io.fits as fits
from astropy.table import Table, vstack, unique

from astropy.time import TimeDelta

# Download HST data

First, we'll read in a list of high-proper motion M dwarfs. [Arjun note: I've downloaded these based on a paper. Should I do this with a MAST query instead? I think I could perhaps do it with astroquery...]

In [4]:
df = pd.read_csv('high_pm_m_dwarfs.csv')

In [5]:
df

,index,recno,RAJ2000,DEJ2000,Name,Comp,2MASS,Conf,l_Sep,Sep,...,Hmag,e_Hmag,Kmag,e_Kmag,n_Kmag,Mass,e_Mass,n_Mass,Memb,SimbadName
0,333,387,269.452125,4.693444,BARNARDS STAR,NaN,17574849+0441405,NaN,NaN,NaN,...,4.834,0.034,4.524,0.020,NaN,0.155,0.014,NaN,s,NAME Barnard's star
1,439,330,217.429250,-62.679639,PROXIMA CEN,C,14294291-6240465,AC,NaN,7803.9000,...,4.835,0.057,4.384,0.033,NaN,0.122,0.014,NaN,s,PROXIMA CEN
2,164,485,336.998125,57.695889,GJ 860,A,22275958+5741453,AB,NaN,3.1840,...,5.038,0.034,4.777,0.029,J,0.268,0.020,c,s,GJ 860 A
3,179,410,280.695583,59.626889,GJ 725,B,18424688+5937374,AB,NaN,12.6200,...,5.197,0.024,5.000,0.023,NaN,0.265,0.014,NaN,s,GJ 725 B
4,308,414,282.455708,-23.836222,GJ 729,NaN,18494929-2350101,NaN,NaN,NaN,...,5.655,0.034,5.370,0.016,NaN,0.174,0.014,NaN,s,GJ 729
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
507,498,437,297.899542,-35.177083,2MA 1951-3510,B,NaN,AB,NaN,1.9000,...,NaN,NaN,NaN,NaN,NaN,0.147,0.014,v,sc,2MASS J19513587-3510375
508,500,445,301.950167,-31.758028,GJ 781.1,C,NaN,AC,NaN,1.0000,...,NaN,NaN,NaN,NaN,NaN,0.199,0.014,v,sc,NaN
509,502,431,296.224167,-23.633139,LP 869-26,B,NaN,AB,NaN,0.8280,...,NaN,NaN,NaN,NaN,NaN,0.152,0.014,v,sc,LP 869-26
510,509,461,313.907375,-14.035583,GJ 810,C,NaN,AC,NaN,0.0648,...,NaN,NaN,NaN,NaN,NaN,0.201,0.021,a,sc,NaN


In [20]:
index = 1

In [21]:

launch_time = Time('1990-04-24 05:11:23.5')

J2000 = Time('2000-1-1 00:00:00.0') # todo: check

years = np.arange(30)
ra = df['RAJ2000'][index] * u.deg
dec = df['DEJ2000'][index] * u.deg

pm_ra_cosdec = df['pmRA'][index] * np.cos(df['DEJ2000'][index]) * u.mas/u.yr
pm_dec = df['pmDE'][index] * u.mas/u.yr

delta_t = TimeDelta(1.0 * u.yr)

c = SkyCoord(ra=ra, dec=dec,
pm_ra_cosdec=pm_ra_cosdec, pm_dec=pm_dec, obstime=J2000)
    
    
for i, year in enumerate(years): # how long since hubble started?
    new_time = launch_time + year * u.year # when we're simulating the observation


    delta_t = new_time - J2000
    c_trans = c.apply_space_motion(dt=delta_t)
    tmax = (new_time + 0.5 * u.year).mjd # todo: is this too wide a range given how fast it's moving?
    tmin = (new_time - 0.5 * u.year).mjd 
    if i == 0:

        obs_table = Observations.query_criteria(dataproduct_type=["image"],
                                           obs_collection='HST',
                                           coordinates=f"{c_trans.ra.value} {c_trans.dec.value}",
                                            radius=f"{5} arcsec",
                                               t_max=[tmin, tmax],
                                               t_min=[tmin, tmax])


        science_obs = obs_table[obs_table['intentType'] != 'calibration']
        public_obs_tables = science_obs[science_obs['dataRights'] == 'PUBLIC']


    else:
        obs_table = Observations.query_criteria(dataproduct_type=["image"],
                                           obs_collection='HST',
                                           coordinates=f"{c_trans.ra.value} {c_trans.dec.value}",
                                            radius=f"{5} arcsec",
                                               t_max=[tmin, tmax],
                                               t_min=[tmin, tmax])

        public_obs_tables = vstack([public_obs_tables,obs_table])
        
        if len(public_obs_tables) == 0:
            continue

        public_obs_tables = unique(public_obs_tables, keys=['obsid', 'objID', 'objID1'])

        public_obs_tables = public_obs_tables[public_obs_tables['intentType'] != 'calibration']
        public_obs_tables = public_obs_tables[public_obs_tables['dataRights'] == 'PUBLIC']



        

In [22]:
obs_table

dataproduct_type,calib_level,obs_collection,obs_id,target_name,s_ra,s_dec,t_min,t_max,t_exptime,wavelength_region,filters,em_min,em_max,target_classification,obs_title,t_obs_release,instrument_name,proposal_pi,proposal_id,proposal_type,project,sequence_number,provenance_name,s_region,jpegURL,dataURL,dataRights,mtFlag,srcDen,intentType,obsid,objID,objID1,distance
str5,int64,str3,str9,str10,float64,float64,float64,float64,float64,str7,str6,float64,float64,str32,str81,float64,str8,str19,str5,str8,str3,int64,str7,str137,str1,str35,str6,bool,float64,str11,str8,str9,str9,float64
image,1,HST,odxa01vnq,V-V645-CEN,217.3847611184,-62.67551535717,58599.15063271991,58599.15312314815,20.1,Optical,MIRVIS,nan,nan,STAR;EMISSION LINE STAR;EXTRA-SO,The Origin and Impact of Flares in the Closest Planetary System- Proxima Centauri,58782.45491897,STIS/CCD,"MacGregor, Meredith",15651,GO,HST,--,CALSTIS,POLYGON 217.38164975 -62.68345993 217.36834954 -62.67841519 217.38790252 -62.66755311 217.40120118 -62.67259599 217.38164975 -62.68345993,--,mast:HST/product/odxa01vnq_raw.fits,PUBLIC,False,nan,science,26227492,122477379,122477379,0.0
image,1,HST,odxa03ikq,V-V645-CEN,217.3847337363,-62.67551307067,58601.136593483796,58601.13908353009,20.1,Optical,MIRVIS,nan,nan,STAR;EMISSION LINE STAR;EXTRA-SO,The Origin and Impact of Flares in the Closest Planetary System- Proxima Centauri,58784.52052074,STIS/CCD,"MacGregor, Meredith",15651,GO,HST,--,CALSTIS,POLYGON 217.38090597 -62.68339146 217.36807397 -62.67809749 217.3885927 -62.66761758 217.40142382 -62.67290967 217.38090597 -62.68339146,--,mast:HST/product/odxa03ikq_raw.fits,PUBLIC,False,nan,science,26227697,122477242,122477242,0.0
image,1,HST,odxa04a4q,V-V645-CEN,217.3847209456,-62.67551193993,58602.06361863426,58602.0661090625,20.1,Optical,MIRVIS,nan,nan,STAR;EMISSION LINE STAR;EXTRA-SO,The Origin and Impact of Flares in the Closest Planetary System- Proxima Centauri,58785.16091428,STIS/CCD,"MacGregor, Meredith",15651,GO,HST,--,CALSTIS,POLYGON 217.38055622 -62.68335424 217.36795368 -62.67794543 217.38891742 -62.66765282 217.40151939 -62.67305974 217.38055622 -62.68335424,--,mast:HST/product/odxa04a4q_raw.fits,PUBLIC,False,nan,science,26227737,122477377,122477377,0.0
image,1,HST,odxa06jrq,V-V645-CEN,217.3846916982,-62.67550920464,58604.182519409726,58604.18500945602,20.1,Optical,MIRVIS,nan,nan,STAR;EMISSION LINE STAR;EXTRA-SO,The Origin and Impact of Flares in the Closest Planetary System- Proxima Centauri,58788.66855308,STIS/CCD,"MacGregor, Meredith",15651,GO,HST,--,CALSTIS,POLYGON 217.37975143 -62.68325612 217.36770012 -62.67758891 217.38966515 -62.66774623 217.40171665 -62.67341156 217.37975143 -62.68325612,--,mast:HST/product/odxa06jrq_raw.fits,PUBLIC,False,nan,science,26227906,122476554,122476554,0.0
image,1,HST,odxa07mgq,V-V645-CEN,217.3846798184,-62.67550803411,58605.04316755787,58605.04566496528,2.272171,Optical,MIRVIS,nan,nan,STAR;EMISSION LINE STAR;EXTRA-SO,The Origin and Impact of Flares in the Closest Planetary System- Proxima Centauri,58674.0,STIS/CCD,"MacGregor, Meredith",15651,GO,HST,--,CALSTIS,POLYGON 217.37942291 -62.68321091 217.36760605 -62.67744066 217.38997035 -62.66778936 217.40178771 -62.67355772 217.37942291 -62.68321091,--,mast:HST/product/odxa07mgq_raw.fits,PUBLIC,False,nan,science,26227993,118297864,118297864,0.0
image,2,HST,odvb9xmaq,DARK,217.2511318882,-62.73608901584,58605.0144630787,58605.01515752315,60.0,Optical,MIRVIS,nan,nan,--,STIS CCD Dark Monitor Part 2,58605.59302072,STIS/CCD,"Branton, Doug",15547,CAL/STIS,HST,--,CALSTIS,POLYGON 217.37941619 -62.68321609 217.36759896 -62.67744601 217.38996256 -62.66779436 217.4017803 -62.67356254 217.37941619 -62.68321609,--,mast:HST/product/odvb9xmaq_flt.fits,PUBLIC,False,nan,calibration,26227996,122212197,122212197,0.0
image,2,HST,odvb9xm9q,DARK,217.2511318882,-62.73608901584,58605.013259375,58605.01395381944,60.0,Optical,MIRVIS,nan,nan,--,STIS CCD Dark Monitor Part 2,58605.59276609,STIS/CCD,"Branton, Doug",15547,CAL/STIS,HST,--,CALSTIS,

In [ ]:
# then for each of the images, will want to check whether there's something there!

# Exercises

# Resources
- HST resources
- M dwarf resources

# About this Notebook

**Author**: Arjun B. Savel (asavel@umd.edu).

**Last updated**: 2022-08-14

# Citations
If you use `astropy`, `astroquery`, or `numpy` for published research, please cite the
authors. Follow these links for more information about citing `astropy`,
`astroquery`, and `numpy`:

* [Citing `astropy`](https://www.astropy.org/acknowledging.html)
* [Citing `astroquery`](https://astroquery.readthedocs.io/en/latest/#astroquery)
* [Citing `numpy`](https://numpy.org/citing-numpy/)

[Top of Page](#top)<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/>
